In [1]:
pip install num2words

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.3/143.3 kB 1.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for docopt: filename=docopt-0.6.2-py2.py3-none-any.whl size=13706 sha256=6a3612bf84aa43f06126d820ca41f1920c99efbfa651f97fc94a34006c52584a
  Stored in directory: /root/.cache/pip/wheels/fc/ab/d4/5da2067ac95b36618c629a5f93f809425700506f72c9732fac
Successfully built docopt


In [2]:
pip install vaderSentiment

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 126.0/126.0 kB 3.7 MB/s eta 0:00:00


In [3]:
import pandas as pd
import numpy as np
from num2words import num2words
from nltk.sentiment import SentimentIntensityAnalyzer
from sklearn.metrics import accuracy_score, classification_report

In [4]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [5]:
df = pd.read_csv('/content/drive/My Drive/Merged_Bank_App_reviews.csv')
df.head()

,review_Id,Bank,review,score,thumbs_Up_Count,review_App_Version,review_time,manual_labels
0,5854e2fd-b0af-44b5-9458-3efeb2369631,Barclays,Not the smoothest or easiest of apps to naviga...,2,28,2.89.0,27/02/2024 12:49,Neutral
1,2bbc0faa-43a2-4255-8035-23772541cb14,Barclays,"The app itself has good features, eg let's you...",3,102,2.88.2,10/02/2024 09:35,Neutral
2,204edfd3-a877-4d32-be56-02c1e27abc44,Barclays,"An easy App to use. Easy to navigate, fast at ...",5,84,2.89.0,20/02/2024 11:33,Positive
3,1181eb84-aa16-48ea-b455-623e05635048,Barclays,Pointless app. I can see how much I owe and pa...,1,0,2.89.0,28/02/2024 14:06,Negative
4,a074686c-acef-41e8-9e73-c0d9cd5dd04b,Barclays,Update. Issue was resolved quickly and I'm rea...,5,2,2.89.0,27/02/2024 16:41,Positive


In [6]:
#CHECK FOR STRUCTURE
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1990 entries, 0 to 1989
Data columns (total 8 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   review_Id           1989 non-null   object
 1   Bank                1990 non-null   object
 2   review              1990 non-null   object
 3   score               1990 non-null   int64 
 4   thumbs_Up_Count     1990 non-null   int64 
 5   review_App_Version  1885 non-null   object
 6    review_time        1990 non-null   object
 7   manual_labels       1990 non-null   object
dtypes: int64(2), object(6)
memory usage: 124.5+ KB


In [7]:
#REPLACE REVIEW IDS WITH SERIAL IDS
serial_numbers = range(101, 2091)

In [8]:
df['review_id'] = serial_numbers
df.head()

,review_Id,Bank,review,score,thumbs_Up_Count,review_App_Version,review_time,manual_labels,review_id
0,5854e2fd-b0af-44b5-9458-3efeb2369631,Barclays,Not the smoothest or easiest of apps to naviga...,2,28,2.89.0,27/02/2024 12:49,Neutral,101
1,2bbc0faa-43a2-4255-8035-23772541cb14,Barclays,"The app itself has good features, eg let's you...",3,102,2.88.2,10/02/2024 09:35,Neutral,102
2,204edfd3-a877-4d32-be56-02c1e27abc44,Barclays,"An easy App to use. Easy to navigate, fast at ...",5,84,2.89.0,20/02/2024 11:33,Positive,103
3,1181eb84-aa16-48ea-b455-623e05635048,Barclays,Pointless app. I can see how much I owe and pa...,1,0,2.89.0,28/02/2024 14:06,Negative,104
4,a074686c-acef-41e8-9e73-c0d9cd5dd04b,Barclays,Update. Issue was resolved quickly and I'm rea...,5,2,2.89.0,27/02/2024 16:41,Positive,105


In [9]:
#Drop column review_id
df.drop('review_Id', axis=1, inplace=True)
df.head()

,Bank,review,score,thumbs_Up_Count,review_App_Version,review_time,manual_labels,review_id
0,Barclays,Not the smoothest or easiest of apps to naviga...,2,28,2.89.0,27/02/2024 12:49,Neutral,101
1,Barclays,"The app itself has good features, eg let's you...",3,102,2.88.2,10/02/2024 09:35,Neutral,102
2,Barclays,"An easy App to use. Easy to navigate, fast at ...",5,84,2.89.0,20/02/2024 11:33,Positive,103
3,Barclays,Pointless app. I can see how much I owe and pa...,1,0,2.89.0,28/02/2024 14:06,Negative,104
4,Barclays,Update. Issue was resolved quickly and I'm rea...,5,2,2.89.0,27/02/2024 16:41,Positive,105


In [10]:
#MOVE THE review_id column
review_id = df.columns[-1]

In [11]:
df = df[[review_id] + [col for col in df.columns if col != review_id]]
df.head()


,review_id,Bank,review,score,thumbs_Up_Count,review_App_Version,review_time,manual_labels
0,101,Barclays,Not the smoothest or easiest of apps to naviga...,2,28,2.89.0,27/02/2024 12:49,Neutral
1,102,Barclays,"The app itself has good features, eg let's you...",3,102,2.88.2,10/02/2024 09:35,Neutral
2,103,Barclays,"An easy App to use. Easy to navigate, fast at ...",5,84,2.89.0,20/02/2024 11:33,Positive
3,104,Barclays,Pointless app. I can see how much I owe and pa...,1,0,2.89.0,28/02/2024 14:06,Negative
4,105,Barclays,Update. Issue was resolved quickly and I'm rea...,5,2,2.89.0,27/02/2024 16:41,Positive


In [12]:
#RENAME 'score' COLUMN with 'rating'
df.rename(columns={'score': 'ratings'}, inplace=True)
df.head()


,review_id,Bank,review,ratings,thumbs_Up_Count,review_App_Version,review_time,manual_labels
0,101,Barclays,Not the smoothest or easiest of apps to naviga...,2,28,2.89.0,27/02/2024 12:49,Neutral
1,102,Barclays,"The app itself has good features, eg let's you...",3,102,2.88.2,10/02/2024 09:35,Neutral
2,103,Barclays,"An easy App to use. Easy to navigate, fast at ...",5,84,2.89.0,20/02/2024 11:33,Positive
3,104,Barclays,Pointless app. I can see how much I owe and pa...,1,0,2.89.0,28/02/2024 14:06,Negative
4,105,Barclays,Update. Issue was resolved quickly and I'm rea...,5,2,2.89.0,27/02/2024 16:41,Positive


In [13]:
#CREATE A RATING LABEL
def rating(score):
    if score >= 4:
        return 'positive'
    elif score == 3:
        return 'neutral'
    else:
        return 'negative'

In [14]:
df['ratings_label'] = df['ratings'].apply(rating)
df.head()

,review_id,Bank,review,ratings,thumbs_Up_Count,review_App_Version,review_time,manual_labels,ratings_label
0,101,Barclays,Not the smoothest or easiest of apps to naviga...,2,28,2.89.0,27/02/2024 12:49,Neutral,negative
1,102,Barclays,"The app itself has good features, eg let's you...",3,102,2.88.2,10/02/2024 09:35,Neutral,neutral
2,103,Barclays,"An easy App to use. Easy to navigate, fast at ...",5,84,2.89.0,20/02/2024 11:33,Positive,positive
3,104,Barclays,Pointless app. I can see how much I owe and pa...,1,0,2.89.0,28/02/2024 14:06,Negative,negative
4,105,Barclays,Update. Issue was resolved quickly and I'm rea...,5,2,2.89.0,27/02/2024 16:41,Positive,positive


In [15]:
#CHECK FOR WHITESPACES
columns_with_whitespace = [col for col in df.columns if ' ' in col]
if columns_with_whitespace:
    print("Columns with white spaces:", columns_with_whitespace)
else:
    print("No columns contain white spaces.")

Columns with white spaces: [' review_time']


In [16]:
#REMOVE WHITSPACES IN COLUMN NAMES
df.rename(columns=lambda x: x.strip(), inplace=True)

In [17]:
#CHECK AGAIN FOR WHITESPACES
columns_with_whitespace = [col for col in df.columns if ' ' in col]
if columns_with_whitespace:
    print("Columns with white spaces:", columns_with_whitespace)
else:
    print("No columns contain white spaces.")

No columns contain white spaces.


In [18]:
#convert review and label attribute to string character, convert review_time to date character
df['review'] = df['review'].astype(str)
df['manual_labels'] = df['manual_labels'].astype(str)
df['ratings_label'] = df['ratings_label'].astype(str)
df['review_time'] = pd.to_datetime(df['review_time'])

<ipython-input-18-5179f7619633>:5: UserWarning: Parsing dates in %d/%m/%Y %H:%M format when dayfirst=False (the default) was specified. Pass `dayfirst=True` or specify a format to silence this warning.
  df['review_time'] = pd.to_datetime(df['review_time'])


In [19]:
#CHECK FOR DATA TYPE
df.dtypes

review_id                      int64
Bank                          object
review                        object
ratings                        int64
thumbs_Up_Count                int64
review_App_Version            object
review_time           datetime64[ns]
manual_labels                 object
ratings_label                 object
dtype: object

In [20]:
#LABEL COUNTS
df['ratings_label'].value_counts()

ratings_label
negative    1182
positive     527
neutral      281
Name: count, dtype: int64

In [21]:
#LABEL COUNTS
df['manual_labels'].value_counts()

manual_labels
Negative    1143
Neutral      486
Positive     361
Name: count, dtype: int64

In [22]:
#CLEANING OF DATA
#check for missing values
df.isna().sum()

review_id               0
Bank                    0
review                  0
ratings                 0
thumbs_Up_Count         0
review_App_Version    105
review_time             0
manual_labels           0
ratings_label           0
dtype: int64

In [23]:
#REPLACE MISSING VALUES USING BACKWARD FILLING
df['review_App_Version'] = df['review_App_Version'].fillna(method='bfill')
df.head()


,review_id,Bank,review,ratings,thumbs_Up_Count,review_App_Version,review_time,manual_labels,ratings_label
0,101,Barclays,Not the smoothest or easiest of apps to naviga...,2,28,2.89.0,2024-02-27 12:49:00,Neutral,negative
1,102,Barclays,"The app itself has good features, eg let's you...",3,102,2.88.2,2024-02-10 09:35:00,Neutral,neutral
2,103,Barclays,"An easy App to use. Easy to navigate, fast at ...",5,84,2.89.0,2024-02-20 11:33:00,Positive,positive
3,104,Barclays,Pointless app. I can see how much I owe and pa...,1,0,2.89.0,2024-02-28 14:06:00,Negative,negative
4,105,Barclays,Update. Issue was resolved quickly and I'm rea...,5,2,2.89.0,2024-02-27 16:41:00,Positive,positive


In [24]:
#check again for missing values
df.isna().sum()

review_id             0
Bank                  0
review                0
ratings               0
thumbs_Up_Count       0
review_App_Version    0
review_time           0
manual_labels         0
ratings_label         0
dtype: int64

In [25]:
#MAKE A COPY
df1 = df.copy()
df1.head()

,review_id,Bank,review,ratings,thumbs_Up_Count,review_App_Version,review_time,manual_labels,ratings_label
0,101,Barclays,Not the smoothest or easiest of apps to naviga...,2,28,2.89.0,2024-02-27 12:49:00,Neutral,negative
1,102,Barclays,"The app itself has good features, eg let's you...",3,102,2.88.2,2024-02-10 09:35:00,Neutral,neutral
2,103,Barclays,"An easy App to use. Easy to navigate, fast at ...",5,84,2.89.0,2024-02-20 11:33:00,Positive,positive
3,104,Barclays,Pointless app. I can see how much I owe and pa...,1,0,2.89.0,2024-02-28 14:06:00,Negative,negative
4,105,Barclays,Update. Issue was resolved quickly and I'm rea...,5,2,2.89.0,2024-02-27 16:41:00,Positive,positive


In [26]:
#REMOVE PUNCTUATIONS EXCEPT (') and (.)
#using regular expressions
import re

In [27]:
def remove_punctuations(text):
    return re.sub(r'[^\w\s\'\.]', '', text)

In [28]:
df1['review'] = df1['review'].apply(remove_punctuations)
df1.head()

,review_id,Bank,review,ratings,thumbs_Up_Count,review_App_Version,review_time,manual_labels,ratings_label
0,101,Barclays,Not the smoothest or easiest of apps to naviga...,2,28,2.89.0,2024-02-27 12:49:00,Neutral,negative
1,102,Barclays,The app itself has good features eg let's you ...,3,102,2.88.2,2024-02-10 09:35:00,Neutral,neutral
2,103,Barclays,An easy App to use. Easy to navigate fast at m...,5,84,2.89.0,2024-02-20 11:33:00,Positive,positive
3,104,Barclays,Pointless app. I can see how much I owe and pa...,1,0,2.89.0,2024-02-28 14:06:00,Negative,negative
4,105,Barclays,Update. Issue was resolved quickly and I'm rea...,5,2,2.89.0,2024-02-27 16:41:00,Positive,positive


In [29]:
#CONVERT review text to lowercase
# Convert text in review attribute to lowercase
df1['review'] = df1['review'].str.lower()
df1.head()

,review_id,Bank,review,ratings,thumbs_Up_Count,review_App_Version,review_time,manual_labels,ratings_label
0,101,Barclays,not the smoothest or easiest of apps to naviga...,2,28,2.89.0,2024-02-27 12:49:00,Neutral,negative
1,102,Barclays,the app itself has good features eg let's you ...,3,102,2.88.2,2024-02-10 09:35:00,Neutral,neutral
2,103,Barclays,an easy app to use. easy to navigate fast at m...,5,84,2.89.0,2024-02-20 11:33:00,Positive,positive
3,104,Barclays,pointless app. i can see how much i owe and pa...,1,0,2.89.0,2024-02-28 14:06:00,Negative,negative
4,105,Barclays,update. issue was resolved quickly and i'm rea...,5,2,2.89.0,2024-02-27 16:41:00,Positive,positive


In [30]:
#REMOVE HTML TAGS AND URLS
#using Regular Expression
def remove_html(text):
    clean_review = re.sub(r'<[^>]*>', '', text)
    return clean_review

In [31]:
df1['review'] = df1['review'].apply(remove_html)
df1.head()

,review_id,Bank,review,ratings,thumbs_Up_Count,review_App_Version,review_time,manual_labels,ratings_label
0,101,Barclays,not the smoothest or easiest of apps to naviga...,2,28,2.89.0,2024-02-27 12:49:00,Neutral,negative
1,102,Barclays,the app itself has good features eg let's you ...,3,102,2.88.2,2024-02-10 09:35:00,Neutral,neutral
2,103,Barclays,an easy app to use. easy to navigate fast at m...,5,84,2.89.0,2024-02-20 11:33:00,Positive,positive
3,104,Barclays,pointless app. i can see how much i owe and pa...,1,0,2.89.0,2024-02-28 14:06:00,Negative,negative
4,105,Barclays,update. issue was resolved quickly and i'm rea...,5,2,2.89.0,2024-02-27 16:41:00,Positive,positive


In [32]:
def remove_urls(text):
    clean_review = re.sub(r'http\S+', '', text)
    return clean_review

In [33]:
df1['review'] = df1['review'].apply(remove_urls)
df1.head()

,review_id,Bank,review,ratings,thumbs_Up_Count,review_App_Version,review_time,manual_labels,ratings_label
0,101,Barclays,not the smoothest or easiest of apps to naviga...,2,28,2.89.0,2024-02-27 12:49:00,Neutral,negative
1,102,Barclays,the app itself has good features eg let's you ...,3,102,2.88.2,2024-02-10 09:35:00,Neutral,neutral
2,103,Barclays,an easy app to use. easy to navigate fast at m...,5,84,2.89.0,2024-02-20 11:33:00,Positive,positive
3,104,Barclays,pointless app. i can see how much i owe and pa...,1,0,2.89.0,2024-02-28 14:06:00,Negative,negative
4,105,Barclays,update. issue was resolved quickly and i'm rea...,5,2,2.89.0,2024-02-27 16:41:00,Positive,positive


In [34]:
#REMOVE SPECIAL CHARACTERS (except ,.') NOTE (-) should always be the last
def remove_special_characters(text):
    pattern = r'[%&@$"#*^~|!)(><:;_=+?/\-]'
    clean_review = re.sub(pattern, '', text)
    return clean_review

In [35]:
df1['review'] = df1['review'].apply(remove_special_characters)
df1.head()

,review_id,Bank,review,ratings,thumbs_Up_Count,review_App_Version,review_time,manual_labels,ratings_label
0,101,Barclays,not the smoothest or easiest of apps to naviga...,2,28,2.89.0,2024-02-27 12:49:00,Neutral,negative
1,102,Barclays,the app itself has good features eg let's you ...,3,102,2.88.2,2024-02-10 09:35:00,Neutral,neutral
2,103,Barclays,an easy app to use. easy to navigate fast at m...,5,84,2.89.0,2024-02-20 11:33:00,Positive,positive
3,104,Barclays,pointless app. i can see how much i owe and pa...,1,0,2.89.0,2024-02-28 14:06:00,Negative,negative
4,105,Barclays,update. issue was resolved quickly and i'm rea...,5,2,2.89.0,2024-02-27 16:41:00,Positive,positive


In [36]:
#CONVERT numbers to words

In [37]:
#Create a function to convert numbers to words
def numbers_to_words(text):
    words = []
    for word in text.split():
        if word.isdigit():
            words.append(num2words(int(word)))
        else:
            words.append(word)
    return ' '.join(words)

In [38]:
df1['review'] = df1['review'].apply(numbers_to_words)
df1

,review_id,Bank,review,ratings,thumbs_Up_Count,review_App_Version,review_time,manual_labels,ratings_label
0,101,Barclays,not the smoothest or easiest of apps to naviga...,2,28,2.89.0,2024-02-27 12:49:00,Neutral,negative
1,102,Barclays,the app itself has good features eg let's you ...,3,102,2.88.2,2024-02-10 09:35:00,Neutral,neutral
2,103,Barclays,an easy app to use. easy to navigate fast at m...,5,84,2.89.0,2024-02-20 11:33:00,Positive,positive
3,104,Barclays,pointless app. i can see how much i owe and pa...,1,0,2.89.0,2024-02-28 14:06:00,Negative,negative
4,105,Barclays,update. issue was resolved quickly and i'm rea...,5,2,2.89.0,2024-02-27 16:41:00,Positive,positive
...,...,...,...,...,...,...,...,...,...
1985,2086,Virginmoney,overall the app is good. however i am dissatis...,4,0,22.13.153,2022-12-14 21:00:00,Neutral,positive
1986,2087,Virginmoney,forces you to use the app in landscape and won...,1,3,23.3.157,2023-04-29 08:43:00,Negative,negative
1987,2088,Virginmoney,extremely confusing a couple of things why not...,1,26,21.10 .9724,2021-04-22 13:31:00,Negative,negative
1988,2089,Virginmoney,unable to put the app on my new phone three di...,2,0,22.6.231,2022-06-19 10:46:00,Negative,negative


In [39]:
import nltk
nltk.download('vader_lexicon')

[nltk_data] Downloading package vader_lexicon to /root/nltk_data...


True

In [40]:
#REMOVE STOP WORDS
from nltk.corpus import stopwords

In [41]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [42]:
print(stopwords.words('english'))

['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so', 'than', '

In [43]:
words_to_remove = ['have', 'has', 'had', 'having', 'do', 'did','i', 'does', 'and', 'but', 'if', 'before', 'after', 'in',
                   'out', 'off', 'again', 'most', 'no', 'nor', 'not', 'only', 'than', 'too', 'very', 's', 't', 'can',
                   'don', "don't", 'should', "should've", 'now', 'd', 'll', 'm', 'o', 're', 've', 'y', 'ain', 'aren',
                   "aren't", 'couldn', "couldn't", 'didn', "didn't", 'doesn', "doesn't", 'hadn', "hadn't", 'hasn', "hasn't",
                   'haven', "haven't", 'isn', "isn't", 'mightn', "mightn't", 'mustn', "mustn't", 'needn', "needn't", 'shan',
                   "shan't", 'shouldn', "shouldn't", 'wasn', "wasn't", 'weren', "weren't", 'won', "won't", 'wouldn',
                   "wouldn't", 'was', 'been', 'being']

In [44]:
stopwords_list = stopwords.words('english')
new_stopwords = [word for word in stopwords_list if word not in words_to_remove]

In [45]:
print(new_stopwords)

['me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'were', 'be', 'doing', 'a', 'an', 'the', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'above', 'below', 'to', 'from', 'up', 'down', 'on', 'over', 'under', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'other', 'some', 'such', 'own', 'same', 'so', 'will', 'just', 'ma']


In [46]:
#FIRSTLY IMPORT word_tokenize before APPLYING remove_stopwords
def remove_stopwords(review):
    tokens = word_tokenize(review)
    filtered_tokens = [word for word in tokens if word.lower() not in new_stopwords]
    filtered_review = ' '.join(filtered_tokens)
    return filtered_review

In [47]:
from nltk.tokenize import word_tokenize
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [48]:
df1['review'] = df1['review'].apply(remove_stopwords)
df1

,review_id,Bank,review,ratings,thumbs_Up_Count,review_App_Version,review_time,manual_labels,ratings_label
0,101,Barclays,not smoothest easiest apps navigate . but choi...,2,28,2.89.0,2024-02-27 12:49:00,Neutral,negative
1,102,Barclays,app has good features eg let 's track spend ca...,3,102,2.88.2,2024-02-10 09:35:00,Neutral,neutral
2,103,Barclays,easy app use . easy navigate fast making trans...,5,84,2.89.0,2024-02-20 11:33:00,Positive,positive
3,104,Barclays,pointless app . i can see much i owe and pay o...,1,0,2.89.0,2024-02-28 14:06:00,Negative,negative
4,105,Barclays,update . issue was resolved quickly and i 'm r...,5,2,2.89.0,2024-02-27 16:41:00,Positive,positive
...,...,...,...,...,...,...,...,...,...
1985,2086,Virginmoney,overall app good . however i dissatisfied foll...,4,0,22.13.153,2022-12-14 21:00:00,Neutral,positive
1986,2087,Virginmoney,forces use app in landscape and wo n't allow s...,1,3,23.3.157,2023-04-29 08:43:00,Negative,negative
1987,2088,Virginmoney,extremely confusing couple things not have int...,1,26,21.10 .9724,2021-04-22 13:31:00,Negative,negative
1988,2089,Virginmoney,unable put app new phone three different peopl...,2,0,22.6.231,2022-06-19 10:46:00,Negative,negative


In [49]:
#MAKE ANOTHER COPY
df2 = df1.copy()
df2.head()

,review_id,Bank,review,ratings,thumbs_Up_Count,review_App_Version,review_time,manual_labels,ratings_label
0,101,Barclays,not smoothest easiest apps navigate . but choi...,2,28,2.89.0,2024-02-27 12:49:00,Neutral,negative
1,102,Barclays,app has good features eg let 's track spend ca...,3,102,2.88.2,2024-02-10 09:35:00,Neutral,neutral
2,103,Barclays,easy app use . easy navigate fast making trans...,5,84,2.89.0,2024-02-20 11:33:00,Positive,positive
3,104,Barclays,pointless app . i can see much i owe and pay o...,1,0,2.89.0,2024-02-28 14:06:00,Negative,negative
4,105,Barclays,update . issue was resolved quickly and i 'm r...,5,2,2.89.0,2024-02-27 16:41:00,Positive,positive


In [50]:
#TOKENIZATION
# Define a function to tokenize text
def tokenize_review(review):
    tokens = word_tokenize(review)
    return tokens

In [51]:
df2['tokens'] = df2['review'].apply(tokenize_review)
df2.head()

,review_id,Bank,review,ratings,thumbs_Up_Count,review_App_Version,review_time,manual_labels,ratings_label,tokens
0,101,Barclays,not smoothest easiest apps navigate . but choi...,2,28,2.89.0,2024-02-27 12:49:00,Neutral,negative,"[not, smoothest, easiest, apps, navigate, ., b..."
1,102,Barclays,app has good features eg let 's track spend ca...,3,102,2.88.2,2024-02-10 09:35:00,Neutral,neutral,"[app, has, good, features, eg, let, 's, track,..."
2,103,Barclays,easy app use . easy navigate fast making trans...,5,84,2.89.0,2024-02-20 11:33:00,Positive,positive,"[easy, app, use, ., easy, navigate, fast, maki..."
3,104,Barclays,pointless app . i can see much i owe and pay o...,1,0,2.89.0,2024-02-28 14:06:00,Negative,negative,"[pointless, app, ., i, can, see, much, i, owe,..."
4,105,Barclays,update . issue was resolved quickly and i 'm r...,5,2,2.89.0,2024-02-27 16:41:00,Positive,positive,"[update, ., issue, was, resolved, quickly, and..."


In [52]:
df2.to_csv('cleaned_total_reviews.csv', index=False)

In [53]:
from google.colab import files

In [54]:
files.download('cleaned_total_reviews.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [55]:
#CREATE A VADER SENTIMENT ANALYZER
analyzer = SentimentIntensityAnalyzer()
sid = SentimentIntensityAnalyzer()

In [56]:
df2['scores'] = df2['tokens'].apply(lambda tokens: sid.polarity_scores(' '.join(tokens)))
df2.head()

,review_id,Bank,review,ratings,thumbs_Up_Count,review_App_Version,review_time,manual_labels,ratings_label,tokens,scores
0,101,Barclays,not smoothest easiest apps navigate . but choi...,2,28,2.89.0,2024-02-27 12:49:00,Neutral,negative,"[not, smoothest, easiest, apps, navigate, ., b...","{'neg': 0.067, 'neu': 0.612, 'pos': 0.321, 'co..."
1,102,Barclays,app has good features eg let 's track spend ca...,3,102,2.88.2,2024-02-10 09:35:00,Neutral,neutral,"[app, has, good, features, eg, let, 's, track,...","{'neg': 0.0, 'neu': 0.839, 'pos': 0.161, 'comp..."
2,103,Barclays,easy app use . easy navigate fast making trans...,5,84,2.89.0,2024-02-20 11:33:00,Positive,positive,"[easy, app, use, ., easy, navigate, fast, maki...","{'neg': 0.075, 'neu': 0.69, 'pos': 0.235, 'com..."
3,104,Barclays,pointless app . i can see much i owe and pay o...,1,0,2.89.0,2024-02-28 14:06:00,Negative,negative,"[pointless, app, ., i, can, see, much, i, owe,...","{'neg': 0.08, 'neu': 0.845, 'pos': 0.075, 'com..."
4,105,Barclays,update . issue was resolved quickly and i 'm r...,5,2,2.89.0,2024-02-27 16:41:00,Positive,positive,"[update, ., issue, was, resolved, quickly, and...","{'neg': 0.19, 'neu': 0.55, 'pos': 0.26, 'compo..."


In [57]:
df2['compound'] = df2['scores'].apply(lambda score: score['compound'])
df2.head()

,review_id,Bank,review,ratings,thumbs_Up_Count,review_App_Version,review_time,manual_labels,ratings_label,tokens,scores,compound
0,101,Barclays,not smoothest easiest apps navigate . but choi...,2,28,2.89.0,2024-02-27 12:49:00,Neutral,negative,"[not, smoothest, easiest, apps, navigate, ., b...","{'neg': 0.067, 'neu': 0.612, 'pos': 0.321, 'co...",0.9549
1,102,Barclays,app has good features eg let 's track spend ca...,3,102,2.88.2,2024-02-10 09:35:00,Neutral,neutral,"[app, has, good, features, eg, let, 's, track,...","{'neg': 0.0, 'neu': 0.839, 'pos': 0.161, 'comp...",0.8399
2,103,Barclays,easy app use . easy navigate fast making trans...,5,84,2.89.0,2024-02-20 11:33:00,Positive,positive,"[easy, app, use, ., easy, navigate, fast, maki...","{'neg': 0.075, 'neu': 0.69, 'pos': 0.235, 'com...",0.8799
3,104,Barclays,pointless app . i can see much i owe and pay o...,1,0,2.89.0,2024-02-28 14:06:00,Negative,negative,"[pointless, app, ., i, can, see, much, i, owe,...","{'neg': 0.08, 'neu': 0.845, 'pos': 0.075, 'com...",-0.0690
4,105,Barclays,update . issue was resolved quickly and i 'm r...,5,2,2.89.0,2024-02-27 16:41:00,Positive,positive,"[update, ., issue, was, resolved, quickly, and...","{'neg': 0.19, 'neu': 0.55, 'pos': 0.26, 'compo...",0.6240


In [58]:
# Define a function to classify sentiments based on VADER's compound score
def vader_classify(tokens):
    # Join the list of tokens into a single string
    text = ' '.join(tokens)
    # Calculate the sentiment scores of the text
    score = analyzer.polarity_scores(text)
    # Classify based on the compound score
    if score['compound'] > 0.80:
        return 'Positive'
    elif score['compound'] < -0.3:
        return 'Negative'
    else:
        return 'Neutral'

In [59]:
# Apply sentiment classification
df2['vader_label'] = df2['tokens'].apply(vader_classify)
df2.head()

,review_id,Bank,review,ratings,thumbs_Up_Count,review_App_Version,review_time,manual_labels,ratings_label,tokens,scores,compound,vader_label
0,101,Barclays,not smoothest easiest apps navigate . but choi...,2,28,2.89.0,2024-02-27 12:49:00,Neutral,negative,"[not, smoothest, easiest, apps, navigate, ., b...","{'neg': 0.067, 'neu': 0.612, 'pos': 0.321, 'co...",0.9549,Positive
1,102,Barclays,app has good features eg let 's track spend ca...,3,102,2.88.2,2024-02-10 09:35:00,Neutral,neutral,"[app, has, good, features, eg, let, 's, track,...","{'neg': 0.0, 'neu': 0.839, 'pos': 0.161, 'comp...",0.8399,Positive
2,103,Barclays,easy app use . easy navigate fast making trans...,5,84,2.89.0,2024-02-20 11:33:00,Positive,positive,"[easy, app, use, ., easy, navigate, fast, maki...","{'neg': 0.075, 'neu': 0.69, 'pos': 0.235, 'com...",0.8799,Positive
3,104,Barclays,pointless app . i can see much i owe and pay o...,1,0,2.89.0,2024-02-28 14:06:00,Negative,negative,"[pointless, app, ., i, can, see, much, i, owe,...","{'neg': 0.08, 'neu': 0.845, 'pos': 0.075, 'com...",-0.0690,Neutral
4,105,Barclays,update . issue was resolved quickly and i 'm r...,5,2,2.89.0,2024-02-27 16:41:00,Positive,positive,"[update, ., issue, was, resolved, quickly, and...","{'neg': 0.19, 'neu': 0.55, 'pos': 0.26, 'compo...",0.6240,Neutral


In [60]:
#CHECK FOR ACCURACY WITH RATINGS LABEL AND MANUAL LABELS


In [61]:
accuracy_score(df2['ratings_label'], df2['vader_label'])
print("Accuracy:", accuracy_score)
print("Classification Report:\n", classification_report(df2['ratings_label'], df2['vader_label']))

Accuracy: <function accuracy_score at 0x7b757404a7a0>
Classification Report:
               precision    recall  f1-score   support

    Negative       0.00      0.00      0.00       0.0
     Neutral       0.00      0.00      0.00       0.0
    Positive       0.00      0.00      0.00       0.0
    negative       0.00      0.00      0.00    1182.0
     neutral       0.00      0.00      0.00     281.0
    positive       0.00      0.00      0.00     527.0

    accuracy                           0.00    1990.0
   macro avg       0.00      0.00      0.00    1990.0
weighted avg       0.00      0.00      0.00    1990.0



/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.

In [62]:
#It didn't work because of the way the labels were written
df2['ratings_label'] = df2['ratings_label'].replace({'negative': 'Negative', 'positive': 'Positive', 'neutral': 'Neutral'})
df2.head()

,review_id,Bank,review,ratings,thumbs_Up_Count,review_App_Version,review_time,manual_labels,ratings_label,tokens,scores,compound,vader_label
0,101,Barclays,not smoothest easiest apps navigate . but choi...,2,28,2.89.0,2024-02-27 12:49:00,Neutral,Negative,"[not, smoothest, easiest, apps, navigate, ., b...","{'neg': 0.067, 'neu': 0.612, 'pos': 0.321, 'co...",0.9549,Positive
1,102,Barclays,app has good features eg let 's track spend ca...,3,102,2.88.2,2024-02-10 09:35:00,Neutral,Neutral,"[app, has, good, features, eg, let, 's, track,...","{'neg': 0.0, 'neu': 0.839, 'pos': 0.161, 'comp...",0.8399,Positive
2,103,Barclays,easy app use . easy navigate fast making trans...,5,84,2.89.0,2024-02-20 11:33:00,Positive,Positive,"[easy, app, use, ., easy, navigate, fast, maki...","{'neg': 0.075, 'neu': 0.69, 'pos': 0.235, 'com...",0.8799,Positive
3,104,Barclays,pointless app . i can see much i owe and pay o...,1,0,2.89.0,2024-02-28 14:06:00,Negative,Negative,"[pointless, app, ., i, can, see, much, i, owe,...","{'neg': 0.08, 'neu': 0.845, 'pos': 0.075, 'com...",-0.0690,Neutral
4,105,Barclays,update . issue was resolved quickly and i 'm r...,5,2,2.89.0,2024-02-27 16:41:00,Positive,Positive,"[update, ., issue, was, resolved, quickly, and...","{'neg': 0.19, 'neu': 0.55, 'pos': 0.26, 'compo...",0.6240,Neutral


In [63]:
#APPLY THE ACCURACY SCORE AGAIN (ratings label)
accuracy = accuracy_score(df2['ratings_label'], df2['vader_label'])
print("Accuracy:", accuracy)
print("Classification Report:\n", classification_report(df2['ratings_label'], df2['vader_label']))

Accuracy: 0.46834170854271356
Classification Report:
               precision    recall  f1-score   support

    Negative       0.81      0.47      0.59      1182
     Neutral       0.15      0.49      0.23       281
    Positive       0.62      0.45      0.52       527

    accuracy                           0.47      1990
   macro avg       0.52      0.47      0.45      1990
weighted avg       0.66      0.47      0.52      1990



In [64]:
#manual labels
accuracy = accuracy_score(df2['manual_labels'], df2['vader_label'])
print("Accuracy:", accuracy)
print("Classification Report:\n", classification_report(df2['manual_labels'], df2['vader_label']))

Accuracy: 0.49246231155778897
Classification Report:
               precision    recall  f1-score   support

    Negative       0.78      0.47      0.59      1143
     Neutral       0.27      0.51      0.35       486
    Positive       0.50      0.54      0.52       361

    accuracy                           0.49      1990
   macro avg       0.52      0.51      0.49      1990
weighted avg       0.61      0.49      0.52      1990



In [65]:
#POS TAGGING
from nltk import pos_tag

In [66]:
import nltk
nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.


True

In [67]:
# Function to apply POS tagging to a tokenized review
def pos_tagging(tokens):
    pos_tags = pos_tag(tokens)
    return pos_tags

In [68]:
df2['pos_tags'] = df2['tokens'].apply(pos_tagging)
df2.head()

,review_id,Bank,review,ratings,thumbs_Up_Count,review_App_Version,review_time,manual_labels,ratings_label,tokens,scores,compound,vader_label,pos_tags
0,101,Barclays,not smoothest easiest apps navigate . but choi...,2,28,2.89.0,2024-02-27 12:49:00,Neutral,Negative,"[not, smoothest, easiest, apps, navigate, ., b...","{'neg': 0.067, 'neu': 0.612, 'pos': 0.321, 'co...",0.9549,Positive,"[(not, RB), (smoothest, JJS), (easiest, JJS), ..."
1,102,Barclays,app has good features eg let 's track spend ca...,3,102,2.88.2,2024-02-10 09:35:00,Neutral,Neutral,"[app, has, good, features, eg, let, 's, track,...","{'neg': 0.0, 'neu': 0.839, 'pos': 0.161, 'comp...",0.8399,Positive,"[(app, NN), (has, VBZ), (good, JJ), (features,..."
2,103,Barclays,easy app use . easy navigate fast making trans...,5,84,2.89.0,2024-02-20 11:33:00,Positive,Positive,"[easy, app, use, ., easy, navigate, fast, maki...","{'neg': 0.075, 'neu': 0.69, 'pos': 0.235, 'com...",0.8799,Positive,"[(easy, JJ), (app, NN), (use, NN), (., .), (ea..."
3,104,Barclays,pointless app . i can see much i owe and pay o...,1,0,2.89.0,2024-02-28 14:06:00,Negative,Negative,"[pointless, app, ., i, can, see, much, i, owe,...","{'neg': 0.08, 'neu': 0.845, 'pos': 0.075, 'com...",-0.0690,Neutral,"[(pointless, NN), (app, NN), (., .), (i, NN), ..."
4,105,Barclays,update . issue was resolved quickly and i 'm r...,5,2,2.89.0,2024-02-27 16:41:00,Positive,Positive,"[update, ., issue, was, resolved, quickly, and...","{'neg': 0.19, 'neu': 0.55, 'pos': 0.26, 'compo...",0.6240,Neutral,"[(update, NN), (., .), (issue, NN), (was, VBD)..."


In [69]:
def modify_sentiment_scores(df2):
    modified_scores = []
    for tokens, pos_tags in zip(df2['tokens'], df2['pos_tags']):
        review_modified_scores = []  # Reset for each review
        for token, pos_tag in zip(tokens, pos_tags):
            sentiment_score = sid.polarity_scores(token)['compound']

            # Modify sentiment score based on POS tag
            if pos_tag[1].startswith('NN'):  # Noun
                # Increase sentiment score for nouns
                sentiment_score *= 1.0
            elif pos_tag[1].startswith('VB'):  # Verb
                # Decrease sentiment score for verbs
                sentiment_score *= 1.0
            elif pos_tag[1].startswith('RB'):  # Verb
                sentiment_score *= 4.0
            elif pos_tag[1].startswith('JJS'):  # Verb
                sentiment_score *= 4.0
            elif pos_tag[1].startswith('NNS'):  # Verb
                sentiment_score *= 2.0
            elif pos_tag[1].startswith('CC'):  # Verb
                sentiment_score *= 1.0
            elif pos_tag[1].startswith('JJR'):  # Verb
                sentiment_score *= 4.0
            elif pos_tag[1].startswith('VBP'):  # Verb
                sentiment_score *= 1.0
            elif pos_tag[1].startswith('JJ'):  # Verb
                sentiment_score *= 3.0
            elif pos_tag[1].startswith('VBN'):  # Verb
                sentiment_score *= 1.0
            elif pos_tag[1].startswith('VBG'):  # Verb
                sentiment_score *= 2.0
            elif pos_tag[1].startswith('CD'):  # Verb
                sentiment_score *= 1.0
            elif pos_tag[1].startswith('IN'):  # Verb
                sentiment_score *= 1.0
            elif pos_tag[1].startswith('VBD'):  # Verb
                sentiment_score *= 1.5
            elif pos_tag[1].startswith('RBR'):  # Verb
                sentiment_score *= 1.5
            # Append the modified sentiment score to review_modified_scores
            review_modified_scores.append(sentiment_score)

        # Check if review_modified_scores is not empty before division
        if review_modified_scores:
            # Calculate the average sentiment score for the review
            avg_score = sum(review_modified_scores) / len(review_modified_scores)
        else:
            # If no sentiment scores are available, assign a default value
            avg_score = 0

        # Append the average sentiment score to modified_scores
        modified_scores.append(avg_score)

    print("Length of modified scores:", len(modified_scores))
    print("Length of DataFrame:", len(df2))

    return modified_scores

In [70]:
df2['modified_scores'] = modify_sentiment_scores(df2)

Length of modified scores: 1990
Length of DataFrame: 1990


In [71]:
#CREATE A SENTIMENT (vader_label_tags)
#CREATE A SENTIMENT
df2['vader_label_tags'] = df2['modified_scores'].apply(lambda score: 'Positive' if score >= 0.05
                                         else ('Negative' if score <= 0 else 'Neutral'))
df2.head()

,review_id,Bank,review,ratings,thumbs_Up_Count,review_App_Version,review_time,manual_labels,ratings_label,tokens,scores,compound,vader_label,pos_tags,modified_scores,vader_label_tags
0,101,Barclays,not smoothest easiest apps navigate . but choi...,2,28,2.89.0,2024-02-27 12:49:00,Neutral,Negative,"[not, smoothest, easiest, apps, navigate, ., b...","{'neg': 0.067, 'neu': 0.612, 'pos': 0.321, 'co...",0.9549,Positive,"[(not, RB), (smoothest, JJS), (easiest, JJS), ...",0.155254,Positive
1,102,Barclays,app has good features eg let 's track spend ca...,3,102,2.88.2,2024-02-10 09:35:00,Neutral,Neutral,"[app, has, good, features, eg, let, 's, track,...","{'neg': 0.0, 'neu': 0.839, 'pos': 0.161, 'comp...",0.8399,Positive,"[(app, NN), (has, VBZ), (good, JJ), (features,...",0.068338,Positive
2,103,Barclays,easy app use . easy navigate fast making trans...,5,84,2.89.0,2024-02-20 11:33:00,Positive,Positive,"[easy, app, use, ., easy, navigate, fast, maki...","{'neg': 0.075, 'neu': 0.69, 'pos': 0.235, 'com...",0.8799,Positive,"[(easy, JJ), (app, NN), (use, NN), (., .), (ea...",0.095758,Positive
3,104,Barclays,pointless app . i can see much i owe and pay o...,1,0,2.89.0,2024-02-28 14:06:00,Negative,Negative,"[pointless, app, ., i, can, see, much, i, owe,...","{'neg': 0.08, 'neu': 0.845, 'pos': 0.075, 'com...",-0.0690,Neutral,"[(pointless, NN), (app, NN), (., .), (i, NN), ...",-0.001626,Negative
4,105,Barclays,update . issue was resolved quickly and i 'm r...,5,2,2.89.0,2024-02-27 16:41:00,Positive,Positive,"[update, ., issue, was, resolved, quickly, and...","{'neg': 0.19, 'neu': 0.55, 'pos': 0.26, 'compo...",0.6240,Neutral,"[(update, NN), (., .), (issue, NN), (was, VBD)...",0.085263,Positive


In [72]:
#Accuracy score (ratings_label)
accuracy = accuracy_score(df2['ratings_label'], df2['vader_label_tags'])
print("Accuracy:", accuracy)
print("Classification Report:\n", classification_report(df2['ratings_label'], df2['vader_label_tags']))

Accuracy: 0.5150753768844221
Classification Report:
               precision    recall  f1-score   support

    Negative       0.84      0.51      0.63      1182
     Neutral       0.16      0.44      0.24       281
    Positive       0.59      0.57      0.58       527

    accuracy                           0.52      1990
   macro avg       0.53      0.51      0.48      1990
weighted avg       0.68      0.52      0.56      1990



In [73]:
#Accuracy score (manual_label)
accuracy = accuracy_score(df2['manual_labels'], df2['vader_label_tags'])
print("Accuracy:", accuracy)
print("Classification Report:\n", classification_report(df2['manual_labels'], df2['vader_label_tags']))

Accuracy: 0.5407035175879397
Classification Report:
               precision    recall  f1-score   support

    Negative       0.84      0.52      0.65      1143
     Neutral       0.30      0.48      0.37       486
    Positive       0.48      0.68      0.56       361

    accuracy                           0.54      1990
   macro avg       0.54      0.56      0.53      1990
weighted avg       0.64      0.54      0.56      1990



In [74]:
#TEXTBLOB
from textblob import TextBlob

In [75]:
df3 = df2.copy()
df3.head()

,review_id,Bank,review,ratings,thumbs_Up_Count,review_App_Version,review_time,manual_labels,ratings_label,tokens,scores,compound,vader_label,pos_tags,modified_scores,vader_label_tags
0,101,Barclays,not smoothest easiest apps navigate . but choi...,2,28,2.89.0,2024-02-27 12:49:00,Neutral,Negative,"[not, smoothest, easiest, apps, navigate, ., b...","{'neg': 0.067, 'neu': 0.612, 'pos': 0.321, 'co...",0.9549,Positive,"[(not, RB), (smoothest, JJS), (easiest, JJS), ...",0.155254,Positive
1,102,Barclays,app has good features eg let 's track spend ca...,3,102,2.88.2,2024-02-10 09:35:00,Neutral,Neutral,"[app, has, good, features, eg, let, 's, track,...","{'neg': 0.0, 'neu': 0.839, 'pos': 0.161, 'comp...",0.8399,Positive,"[(app, NN), (has, VBZ), (good, JJ), (features,...",0.068338,Positive
2,103,Barclays,easy app use . easy navigate fast making trans...,5,84,2.89.0,2024-02-20 11:33:00,Positive,Positive,"[easy, app, use, ., easy, navigate, fast, maki...","{'neg': 0.075, 'neu': 0.69, 'pos': 0.235, 'com...",0.8799,Positive,"[(easy, JJ), (app, NN), (use, NN), (., .), (ea...",0.095758,Positive
3,104,Barclays,pointless app . i can see much i owe and pay o...,1,0,2.89.0,2024-02-28 14:06:00,Negative,Negative,"[pointless, app, ., i, can, see, much, i, owe,...","{'neg': 0.08, 'neu': 0.845, 'pos': 0.075, 'com...",-0.0690,Neutral,"[(pointless, NN), (app, NN), (., .), (i, NN), ...",-0.001626,Negative
4,105,Barclays,update . issue was resolved quickly and i 'm r...,5,2,2.89.0,2024-02-27 16:41:00,Positive,Positive,"[update, ., issue, was, resolved, quickly, and...","{'neg': 0.19, 'neu': 0.55, 'pos': 0.26, 'compo...",0.6240,Neutral,"[(update, NN), (., .), (issue, NN), (was, VBD)...",0.085263,Positive


In [76]:
columns_to_drop = ['scores', 'compound','vader_label', 'modified_scores', 'vader_label_tags']
df3.drop(columns=columns_to_drop, inplace=True)
df3.head()

,review_id,Bank,review,ratings,thumbs_Up_Count,review_App_Version,review_time,manual_labels,ratings_label,tokens,pos_tags
0,101,Barclays,not smoothest easiest apps navigate . but choi...,2,28,2.89.0,2024-02-27 12:49:00,Neutral,Negative,"[not, smoothest, easiest, apps, navigate, ., b...","[(not, RB), (smoothest, JJS), (easiest, JJS), ..."
1,102,Barclays,app has good features eg let 's track spend ca...,3,102,2.88.2,2024-02-10 09:35:00,Neutral,Neutral,"[app, has, good, features, eg, let, 's, track,...","[(app, NN), (has, VBZ), (good, JJ), (features,..."
2,103,Barclays,easy app use . easy navigate fast making trans...,5,84,2.89.0,2024-02-20 11:33:00,Positive,Positive,"[easy, app, use, ., easy, navigate, fast, maki...","[(easy, JJ), (app, NN), (use, NN), (., .), (ea..."
3,104,Barclays,pointless app . i can see much i owe and pay o...,1,0,2.89.0,2024-02-28 14:06:00,Negative,Negative,"[pointless, app, ., i, can, see, much, i, owe,...","[(pointless, NN), (app, NN), (., .), (i, NN), ..."
4,105,Barclays,update . issue was resolved quickly and i 'm r...,5,2,2.89.0,2024-02-27 16:41:00,Positive,Positive,"[update, ., issue, was, resolved, quickly, and...","[(update, NN), (., .), (issue, NN), (was, VBD)..."


In [77]:
# Function to analyze sentiment from text using TextBlob
def analyze_sentiment(text):
    analysis = TextBlob(text)
    return pd.Series([analysis.sentiment.polarity, analysis.sentiment.subjectivity])

In [78]:
df3['tokens'] = df3['tokens'].apply(lambda tokens: ' '.join(tokens))

In [79]:
# Apply sentiment analysis on the formatted string
df3[['polarity', 'subjectivity']] = df3['tokens'].apply(analyze_sentiment)

In [80]:
df3.head()

,review_id,Bank,review,ratings,thumbs_Up_Count,review_App_Version,review_time,manual_labels,ratings_label,tokens,pos_tags,polarity,subjectivity
0,101,Barclays,not smoothest easiest apps navigate . but choi...,2,28,2.89.0,2024-02-27 12:49:00,Neutral,Negative,not smoothest easiest apps navigate . but choi...,"[(not, RB), (smoothest, JJS), (easiest, JJS), ...",0.300000,0.376667
1,102,Barclays,app has good features eg let 's track spend ca...,3,102,2.88.2,2024-02-10 09:35:00,Neutral,Neutral,app has good features eg let 's track spend ca...,"[(app, NN), (has, VBZ), (good, JJ), (features,...",0.235926,0.440741
2,103,Barclays,easy app use . easy navigate fast making trans...,5,84,2.89.0,2024-02-20 11:33:00,Positive,Positive,easy app use . easy navigate fast making trans...,"[(easy, JJ), (app, NN), (use, NN), (., .), (ea...",0.295833,0.557407
3,104,Barclays,pointless app . i can see much i owe and pay o...,1,0,2.89.0,2024-02-28 14:06:00,Negative,Negative,pointless app . i can see much i owe and pay o...,"[(pointless, NN), (app, NN), (., .), (i, NN), ...",0.094444,0.316667
4,105,Barclays,update . issue was resolved quickly and i 'm r...,5,2,2.89.0,2024-02-27 16:41:00,Positive,Positive,update . issue was resolved quickly and i 'm r...,"[(update, NN), (., .), (issue, NN), (was, VBD)...",0.266667,0.616667


In [81]:
# function to analyze the reviews
def analysis(polarity):
    if polarity < 0:
        return 'Negative'
    elif polarity > 0.20:
        return 'Positive'
    else:
        return 'Neutral'

In [82]:
df3['textblob_label'] = df3['polarity'].apply(analysis)
df3.head()

,review_id,Bank,review,ratings,thumbs_Up_Count,review_App_Version,review_time,manual_labels,ratings_label,tokens,pos_tags,polarity,subjectivity,textblob_label
0,101,Barclays,not smoothest easiest apps navigate . but choi...,2,28,2.89.0,2024-02-27 12:49:00,Neutral,Negative,not smoothest easiest apps navigate . but choi...,"[(not, RB), (smoothest, JJS), (easiest, JJS), ...",0.300000,0.376667,Positive
1,102,Barclays,app has good features eg let 's track spend ca...,3,102,2.88.2,2024-02-10 09:35:00,Neutral,Neutral,app has good features eg let 's track spend ca...,"[(app, NN), (has, VBZ), (good, JJ), (features,...",0.235926,0.440741,Positive
2,103,Barclays,easy app use . easy navigate fast making trans...,5,84,2.89.0,2024-02-20 11:33:00,Positive,Positive,easy app use . easy navigate fast making trans...,"[(easy, JJ), (app, NN), (use, NN), (., .), (ea...",0.295833,0.557407,Positive
3,104,Barclays,pointless app . i can see much i owe and pay o...,1,0,2.89.0,2024-02-28 14:06:00,Negative,Negative,pointless app . i can see much i owe and pay o...,"[(pointless, NN), (app, NN), (., .), (i, NN), ...",0.094444,0.316667,Neutral
4,105,Barclays,update . issue was resolved quickly and i 'm r...,5,2,2.89.0,2024-02-27 16:41:00,Positive,Positive,update . issue was resolved quickly and i 'm r...,"[(update, NN), (., .), (issue, NN), (was, VBD)...",0.266667,0.616667,Positive


In [83]:
#ACCURACY WITH (ratings_label)
accuracy = accuracy_score(df3['ratings_label'], df3['textblob_label'])
print("Accuracy:", accuracy)
print("Classification Report:\n", classification_report(df3['ratings_label'], df3['textblob_label']))

Accuracy: 0.4984924623115578
Classification Report:
               precision    recall  f1-score   support

    Negative       0.83      0.50      0.62      1182
     Neutral       0.18      0.53      0.26       281
    Positive       0.60      0.48      0.53       527

    accuracy                           0.50      1990
   macro avg       0.53      0.50      0.47      1990
weighted avg       0.67      0.50      0.55      1990



In [84]:
#POS_TAGS
def analyze_sentiment_with_pos(text):
    blob = TextBlob(text)
    total_sentiment = 0
    total_tags = 0
    for word, pos_tag in blob.tags:
        word_sentiment = TextBlob(word).sentiment.polarity
        # Modify sentiment score based on POS tag
        if pos_tag[1].startswith('NN'):  # Noun
              # Increase sentiment score for nouns
              weight = 1.0
        elif pos_tag[1].startswith('VB'):  # Verb
            # Decrease sentiment score for verbs
            weight = 1.0
        elif pos_tag[1].startswith('RB'):  # Verb
            weight = 4.0
        elif pos_tag[1].startswith('JJS'):  # Verb
            weight = 4.0
        elif pos_tag[1].startswith('NNS'):  # Verb
            weight = 2.0
        elif pos_tag[1].startswith('CC'):  # Verb
            weight = 1.0
        elif pos_tag[1].startswith('JJR'):  # Verb
            weight = 4.0
        elif pos_tag[1].startswith('VBP'):  # Verb
            weight = 1.0
        elif pos_tag[1].startswith('JJ'):  # Verb
            weight = 3.0
        elif pos_tag[1].startswith('VBN'):  # Verb
            weight = 1.0
        elif pos_tag[1].startswith('VBG'):  # Verb
            weight = 2.0
        elif pos_tag[1].startswith('CD'):  # Verb
            weight = 1.0
        elif pos_tag[1].startswith('IN'):  # Verb
            weight = 1.0
        elif pos_tag[1].startswith('VBD'):  # Verb
            weight = 1.5
        elif pos_tag[1].startswith('RBR'):  # Verb
            weight = 1.5
        else:
            weight = 1.0  # Default case if none of the above

        total_sentiment += word_sentiment * weight
        total_tags += weight

    # Normalize the total sentiment by the sum of weights used
    normalized_sentiment = total_sentiment / total_tags if total_tags != 0 else 0
    return normalized_sentiment

In [85]:
df3['scores1'] = df3['tokens'].apply(analyze_sentiment_with_pos)
df3.head()

,review_id,Bank,review,ratings,thumbs_Up_Count,review_App_Version,review_time,manual_labels,ratings_label,tokens,pos_tags,polarity,subjectivity,textblob_label,scores1
0,101,Barclays,not smoothest easiest apps navigate . but choi...,2,28,2.89.0,2024-02-27 12:49:00,Neutral,Negative,not smoothest easiest apps navigate . but choi...,"[(not, RB), (smoothest, JJS), (easiest, JJS), ...",0.300000,0.376667,Positive,0.028302
1,102,Barclays,app has good features eg let 's track spend ca...,3,102,2.88.2,2024-02-10 09:35:00,Neutral,Neutral,app has good features eg let 's track spend ca...,"[(app, NN), (has, VBZ), (good, JJ), (features,...",0.235926,0.440741,Positive,0.041358
2,103,Barclays,easy app use . easy navigate fast making trans...,5,84,2.89.0,2024-02-20 11:33:00,Positive,Positive,easy app use . easy navigate fast making trans...,"[(easy, JJ), (app, NN), (use, NN), (., .), (ea...",0.295833,0.557407,Positive,0.050223
3,104,Barclays,pointless app . i can see much i owe and pay o...,1,0,2.89.0,2024-02-28 14:06:00,Negative,Negative,pointless app . i can see much i owe and pay o...,"[(pointless, NN), (app, NN), (., .), (i, NN), ...",0.094444,0.316667,Neutral,0.011333
4,105,Barclays,update . issue was resolved quickly and i 'm r...,5,2,2.89.0,2024-02-27 16:41:00,Positive,Positive,update . issue was resolved quickly and i 'm r...,"[(update, NN), (., .), (issue, NN), (was, VBD)...",0.266667,0.616667,Positive,0.057576


In [86]:
df3.to_csv('new_textblob_vader.csv', index=False)

In [87]:
from google.colab import files

In [88]:
files.download('new_textblob_vader.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [89]:
# function to analyze the reviews
def analysis1(scores1):
    if scores1 < 0.002:
        return 'Negative'
    elif scores1 > 0.03:
        return 'Positive'
    else:
        return 'Neutral'

In [90]:
df3['textblob_label_tags'] = df3['scores1'].apply(analysis1)
df3.head()

,review_id,Bank,review,ratings,thumbs_Up_Count,review_App_Version,review_time,manual_labels,ratings_label,tokens,pos_tags,polarity,subjectivity,textblob_label,scores1,textblob_label_tags
0,101,Barclays,not smoothest easiest apps navigate . but choi...,2,28,2.89.0,2024-02-27 12:49:00,Neutral,Negative,not smoothest easiest apps navigate . but choi...,"[(not, RB), (smoothest, JJS), (easiest, JJS), ...",0.300000,0.376667,Positive,0.028302,Neutral
1,102,Barclays,app has good features eg let 's track spend ca...,3,102,2.88.2,2024-02-10 09:35:00,Neutral,Neutral,app has good features eg let 's track spend ca...,"[(app, NN), (has, VBZ), (good, JJ), (features,...",0.235926,0.440741,Positive,0.041358,Positive
2,103,Barclays,easy app use . easy navigate fast making trans...,5,84,2.89.0,2024-02-20 11:33:00,Positive,Positive,easy app use . easy navigate fast making trans...,"[(easy, JJ), (app, NN), (use, NN), (., .), (ea...",0.295833,0.557407,Positive,0.050223,Positive
3,104,Barclays,pointless app . i can see much i owe and pay o...,1,0,2.89.0,2024-02-28 14:06:00,Negative,Negative,pointless app . i can see much i owe and pay o...,"[(pointless, NN), (app, NN), (., .), (i, NN), ...",0.094444,0.316667,Neutral,0.011333,Neutral
4,105,Barclays,update . issue was resolved quickly and i 'm r...,5,2,2.89.0,2024-02-27 16:41:00,Positive,Positive,update . issue was resolved quickly and i 'm r...,"[(update, NN), (., .), (issue, NN), (was, VBD)...",0.266667,0.616667,Positive,0.057576,Positive


In [91]:
#ACCURACY WITH (ratings_label)
accuracy = accuracy_score(df3['ratings_label'], df3['textblob_label_tags'])
print("Accuracy:", accuracy)
print("Classification Report:\n", classification_report(df3['ratings_label'], df3['textblob_label']))

Accuracy: 0.4834170854271357
Classification Report:
               precision    recall  f1-score   support

    Negative       0.83      0.50      0.62      1182
     Neutral       0.18      0.53      0.26       281
    Positive       0.60      0.48      0.53       527

    accuracy                           0.50      1990
   macro avg       0.53      0.50      0.47      1990
weighted avg       0.67      0.50      0.55      1990



In [92]:
#ACCURACY WITH (manual_label)
accuracy = accuracy_score(df3['manual_labels'], df3['textblob_label_tags'])
print("Accuracy:", accuracy)
print("Classification Report:\n", classification_report(df3['manual_labels'], df3['textblob_label']))

Accuracy: 0.5120603015075377
Classification Report:
               precision    recall  f1-score   support

    Negative       0.82      0.51      0.63      1143
     Neutral       0.30      0.53      0.39       486
    Positive       0.48      0.57      0.52       361

    accuracy                           0.53      1990
   macro avg       0.54      0.54      0.51      1990
weighted avg       0.63      0.53      0.55      1990



In [93]:
#ACCURACY WITH (manual_label)
accuracy = accuracy_score(df3['manual_labels'], df3['textblob_label'])
print("Accuracy:", accuracy)
print("Classification Report:\n", classification_report(df3['manual_labels'], df3['textblob_label']))

Accuracy: 0.5271356783919598
Classification Report:
               precision    recall  f1-score   support

    Negative       0.82      0.51      0.63      1143
     Neutral       0.30      0.53      0.39       486
    Positive       0.48      0.57      0.52       361

    accuracy                           0.53      1990
   macro avg       0.54      0.54      0.51      1990
weighted avg       0.63      0.53      0.55      1990

